In [1]:
import heapq
import itertools
import random
from dataclasses import dataclass
from typing import List

In [2]:
@dataclass
class Event:
    time: int


@dataclass
class OrderArrival(Event):
    side: str
    price: float | None
    qty: int


@dataclass
class OrderCancel(Event):
    order_id: int


@dataclass
class MarketClose(Event):
    pass
@dataclass
class Order:
    order_id: int
    side: str            # "buy" or "sell"
    price: float | None  # None for MARKET
    qty: int
    remaining_qty: int
    timestamp: int
    status: str = "NEW"


@dataclass
class Trade:
    price: float
    qty: int
    buyer_id: int
    seller_id: int
    timestamp: int


In [3]:
order_id_gen = itertools.count(1)
clock = itertools.count(1)


In [4]:
class OrderBook:
    def __init__(self):
        self.bids = []   # max-heap: (-price, timestamp, order_id, order)
        self.asks = []   # min-heap: (price, timestamp, order_id, order)
        self.trades: List[Trade] = []
        self.order_lookup = {}
        
    def add_order(self, side, price, qty):
        order = Order(
            order_id=next(order_id_gen),
            side=side,
            price=price,
            qty=qty,
            remaining_qty=qty,
            timestamp=next(clock)
        )
        self.order_lookup[order.order_id] = order
        self.match(order)

        if order.remaining_qty > 0 and price is not None:
            self._add_to_book(order)

    def _add_to_book(self, order: Order):
        if order.side == "buy":
            heapq.heappush(
                self.bids,
                (-order.price, order.timestamp, order.order_id, order)
            )
        else:
            heapq.heappush(
                self.asks,
                (order.price, order.timestamp, order.order_id, order)
            )


    def match(self, incoming: Order):
        while incoming.remaining_qty > 0:
            if incoming.side == "buy":
                if not self.asks:
                    break

                best_price, _, _, resting = self.asks[0]

                if incoming.price is not None and incoming.price < best_price:
                    break

            else:  # incoming sell
                if not self.bids:
                    break

                best_price, _, _, resting = self.bids[0]
                best_price = -best_price

                if incoming.price is not None and incoming.price > best_price:
                    break

            executed_qty = min(incoming.remaining_qty, resting.remaining_qty)
            trade_price = resting.price

            buyer_id = incoming.order_id if incoming.side == "buy" else resting.order_id
            seller_id = resting.order_id if incoming.side == "buy" else incoming.order_id

            self.trades.append(
                Trade(
                    price=trade_price,
                    qty=executed_qty,
                    buyer_id=buyer_id,
                    seller_id=seller_id,
                    timestamp=next(clock)
                )
            )

            incoming.remaining_qty -= executed_qty
            resting.remaining_qty -= executed_qty

            if resting.remaining_qty == 0:
                resting.status = "FILLED"
                if resting.side == "buy":
                    heapq.heappop(self.bids)
                else:
                    heapq.heappop(self.asks)
            else:
                resting.status = "PARTIALLY_FILLED"

        incoming.status = (
            "FILLED" if incoming.remaining_qty == 0 else "PARTIALLY_FILLED"
        )


In [5]:
    def print_book(book: OrderBook):
        print("\nASKS:")
        for p, _, _, o in sorted(book.asks):
            print(f"Price {p}, Qty {o.remaining_qty}")

        print("\nBIDS:")
        for p, _, _, o in sorted(book.bids):
            print(f"Price {-p}, Qty {o.remaining_qty}")

In [6]:
event_seq = itertools.count(1)


In [7]:
class MarketEngine:
    def __init__(self, order_book):
        self.time = 0
        self.event_queue = []   # heap of (time, seq, event)
        self.order_book = order_book
        self.running = True

    def push_event(self, event: Event):
        heapq.heappush(
            self.event_queue,
            (event.time, next(event_seq), event)
        )

    def run(self):
        while self.event_queue and self.running:
            event_time, _, event = heapq.heappop(self.event_queue)
            
            self.time = event_time

            if isinstance(event, OrderArrival):
                self.handle_order(event)

            elif isinstance(event, OrderCancel):
                self.handle_cancel(event)

            elif isinstance(event, MarketClose):
                self.running = False
                
    def handle_order(self, event: OrderArrival):
        self.order_book.add_order(
            side=event.side,
            price=event.price,
            qty=event.qty
        )

    def handle_cancel(self, event: OrderCancel):
        order_id = event.order_id

        order = self.order_lookup.get(order_id)

        if order is None:
            return  

        if order.status in ("FILLED", "CANCELLED"):
            return

        order.status = "CANCELLED"


In [8]:
def send_order(engine, now, side, price, qty, latency):
    arrival_time = now + latency
    engine.push_event(
        OrderArrival(
            time=arrival_time,
            side=side,
            price=price,
            qty=qty
        )
    )


In [9]:
book = OrderBook()
engine = MarketEngine(book)


In [10]:
random.seed(42)

# Initial time
t0 = 0

send_order(engine, t0, "sell", 101, 10, latency=1)
send_order(engine, t0, "sell", 102, 20, latency=1)
send_order(engine, t0, "sell", 103, 30, latency=1)

send_order(engine, t0, "buy", None, 60, latency=2)

engine.push_event(MarketClose(time=10))


In [11]:
engine.run()


In [12]:
print("TRADES:")
for t in book.trades:
    print(f"Time={t.timestamp} Price={t.price} Qty={t.qty}")


TRADES:
Time=5 Price=101 Qty=10
Time=6 Price=102 Qty=20
Time=7 Price=103 Qty=30
